In [0]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [284]:
df = pd.read_csv('./UCI_data.csv')
df.pop('date')
df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,TARGET_energy
0,22.200000,39.500000,20.566667,37.656667,22.230000,37.030000,22.318571,36.610000,20.633333,62.166667,8.823333,13.296667,21.00,29.318571,22.23,38.463333,19.926667,33.90,9.70,766.100000,65.5,3.500000,40.000000,3.350000,24.061869,24.061869,60
1,20.356667,37.126667,17.566667,40.230000,20.890000,37.663333,18.700000,36.260000,18.463333,43.560000,1.230000,78.963333,18.39,32.290000,21.00,43.400000,18.390000,41.09,0.30,740.333333,99.0,1.000000,41.333333,0.100000,4.622052,4.622052,50
2,20.926667,38.790000,21.100000,35.526667,21.600000,36.290000,21.000000,34.826667,18.100000,46.126667,6.033333,37.300000,21.29,27.566667,21.70,32.447500,19.390000,38.76,4.40,768.466667,72.0,6.000000,22.666667,-0.266667,5.635898,5.635898,80
3,18.290000,38.900000,17.290000,39.260000,18.390000,39.326667,16.100000,38.790000,16.100000,47.700000,4.595000,94.945000,16.20,33.590000,17.78,41.663333,15.800000,39.20,3.35,760.600000,82.0,5.500000,41.000000,0.500000,49.216445,49.216445,40
4,22.290000,42.333333,21.600000,40.433333,22.666667,43.363333,19.100000,40.900000,19.290000,50.745000,2.530000,92.190000,20.00,40.000000,23.10,46.090000,18.890000,43.73,3.20,738.900000,88.0,7.333333,56.000000,1.400000,47.617579,47.617579,60


In [285]:
df.dtypes

T1               float64
RH_1             float64
T2               float64
RH_2             float64
T3               float64
RH_3             float64
T4               float64
RH_4             float64
T5               float64
RH_5             float64
T6               float64
RH_6             float64
T7               float64
RH_7             float64
T8               float64
RH_8             float64
T9               float64
RH_9             float64
T_out            float64
Press_mm_hg      float64
RH_out           float64
Windspeed        float64
Visibility       float64
Tdewpoint        float64
rv1              float64
rv2              float64
TARGET_energy      int64
dtype: object

In [286]:
df.isnull().sum()

T1               0
RH_1             0
T2               0
RH_2             0
T3               0
RH_3             0
T4               0
RH_4             0
T5               0
RH_5             0
T6               0
RH_6             0
T7               0
RH_7             0
T8               0
RH_8             0
T9               0
RH_9             0
T_out            0
Press_mm_hg      0
RH_out           0
Windspeed        0
Visibility       0
Tdewpoint        0
rv1              0
rv2              0
TARGET_energy    0
dtype: int64

In [287]:
train_data = df[1:1001]
test_data = df[1002:2002]
print(f'Training data: {len(train_data)} Test data: {len(test_data)} ')

Training data: 1000 Test data: 1000 


In [288]:
# Columns for temperature data
col_temperature = ["T1","T2","T3","T4","T5","T6","T7","T8","T9"]
# Columns for humidity data
col_humidity = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_6","RH_7","RH_8","RH_9"]
mean_T = []
mean_RH = []
for index, row in train_data.iterrows():
    mean_T.append(row[col_temperature].mean())
    mean_RH.append(row[col_humidity].mean())
for i in col_temperature:
  train_data.pop(i)
for i in col_humidity:
  train_data.pop(i)
train_data['MEAN_T'] = mean_T
train_data['MEAN_RH'] = mean_RH

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [289]:
target = train_data.pop('TARGET_energy')
dataset = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))
train_data

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,MEAN_T,MEAN_RH
1,0.300000,740.333333,99.000000,1.000000,41.333333,0.100000,4.622052,4.622052,17.220741,43.398148
2,4.400000,768.466667,72.000000,6.000000,22.666667,-0.266667,5.635898,5.635898,19.015556,36.403796
3,3.350000,760.600000,82.000000,5.500000,41.000000,0.500000,49.216445,49.216445,15.616111,45.930556
4,3.200000,738.900000,88.000000,7.333333,56.000000,1.400000,47.617579,47.617579,18.829630,48.865000
5,4.633333,770.866667,52.333333,2.333333,40.000000,-4.333333,16.431218,16.431218,18.784815,36.035185
...,...,...,...,...,...,...,...,...,...,...
996,3.400000,750.200000,90.000000,5.000000,27.000000,1.900000,3.557701,3.557701,16.525185,51.551852
997,11.700000,759.600000,75.000000,3.000000,29.000000,7.300000,17.066852,17.066852,22.690741,38.202222
998,17.733333,751.766667,80.666667,2.000000,40.000000,14.366667,1.802695,1.802695,24.091679,44.821473
999,11.550000,756.050000,88.333333,10.000000,37.833333,9.616667,27.987976,27.987976,19.262926,53.274704


In [290]:
for f, t in dataset.take(5):
  print ('Features: {}, Target: {}'.format(f, t))

Features: [3.00000000e-01 7.40333333e+02 9.90000000e+01 1.00000000e+00
 4.13333333e+01 1.00000000e-01 4.62205205e+00 4.62205205e+00
 1.72207407e+01 4.33981481e+01], Target: 50
Features: [ 4.40000000e+00  7.68466667e+02  7.20000000e+01  6.00000000e+00
  2.26666667e+01 -2.66666667e-01  5.63589782e+00  5.63589782e+00
  1.90155556e+01  3.64037963e+01], Target: 80
Features: [3.35000000e+00 7.60600000e+02 8.20000000e+01 5.50000000e+00
 4.10000000e+01 5.00000000e-01 4.92164449e+01 4.92164449e+01
 1.56161111e+01 4.59305556e+01], Target: 40
Features: [  3.2        738.9         88.           7.33333333  56.
   1.4         47.61757874  47.61757874  18.82962963  48.865     ], Target: 60
Features: [  4.63333333 770.86666667  52.33333333   2.33333333  40.
  -4.33333333  16.4312178   16.4312178   18.78481481  36.03518519], Target: 50


In [0]:
train_dataset = dataset.shuffle(len(train_data)).batch(1)

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])  

In [293]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 12129.2637 - accuracy: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9302.0234 - accuracy: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9107.3506 - accuracy: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9228.2080 - accuracy: 0.0000e+00
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9184.4639 - accuracy: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9021.5225 - accuracy: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9070.2910 - accuracy: 0.0000e+00
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9096.9512 - accuracy: 0.0000e+00
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9133.9697 - accuracy: 0.0000e+00
Epoch 10/10
1000/1